## goal of this notebook
The project is to ask questions like "what is the largest city in america". Thus we need to: 
- find the adj-noun pair (large - city)
- maybe also find qualifier (*american* city)

## how to do that
1. predetermine "words" (i.e. nouns)
2. find the most frequent adjs in front of that word by searching through sentences in a corpus

## what do we got
Tried quora questions as well as webtext and brown corpus in nltk tool kit. Quora questions are the best. 
Results in "adj-noun.p".
However, unable to distinguish degradable VS absolute adjectives. Need that to produce valid questions!!

In [1]:
from nltk.corpus import webtext
import csv
import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from IPython.core.debugger import set_trace
import collections
import pickle
tokenizer = RegexpTokenizer(r'\w+')
cachedStopWords = stopwords.words("english")

In [100]:
words = ['mountain','population','company','area','income','employment','city']
topics = ['country','income','animal','plant','city']

### use the quora question pair corpus? -- can get the best results. 

In [ ]:
qrsents = pickle.load(open('quorasents.p','rb'))
qresults = searchsent(qrsents,words,topics, 'quora_sents.p')

In [103]:
freas = []
for kw in range(len(words)):
    freas.append(reqadj([qresults[0][kw]]))
    print(words[kw])
    print(freas[kw])
pickle.dump([words,freas],'adj-noun.p')

mountain
['highest' 'biggest' 'largest']
population
['Indian' 'Muslim' 'human' 'general' 'Chinese' 'small' 'Jewish' 'low'
 'growing' 'largest' 'large' 'rural' 'male' 'lesser' 'heavy' 'higher'
 'smallest' 'current' 'civilian' 'biggest' 'Russian' 'Hindu' 'increasing'
 'young']
company
['best' 'good' 'limited' 'private' 'new' 'big' 'Indian' 'pharmaceutical'
 'small' 'current' 'public' 'better' 'large' 'mobile' 'Chinese'
 'registered' 'international' 'owned' 'financial' 'foreign' 'rental'
 'successful' 'Japanese' 'listed' 'reliable' 'top' 'oldest' 'regular'
 'Canadian' 'different' 'Private' 'bigger' 'corporate' 'first' 'held'
 'worst' 'leading' 'profitable' 'legitimate' 'expensive' 'mechanical'
 'related' 'offshore' 'normal' 'single' 'automotive' 'specific' 'analytic'
 'valuable' 'valid' 'industrial' 'lost']
area
['surface' 'best' 'rural' 'public' 'light' 'local' 'residential' 'urban'
 'pubic' 'good' 'general' 'safe' 'small' 'much' 'geographical' 'remote'
 'expensive' 'sectional' 'large' '

TypeError: file must have a 'write' attribute

### just using weights in word net? too limited

In [96]:
wn.synset('weight.n.01').attributes()

[Synset('heavy.a.01'), Synset('light.a.01')]

### try the brown corpus? -- still not large enough, missing a lot.

In [63]:
from nltk.corpus import brown
results=searchsent(brown.sents(),words,topics, 'brown_sents.p',doprint=False)

freas = []
for kw in range(len(words)):
    freas.append(reqadj([results[0][kw]]))
    print(words[kw])
    print(freas[kw])

strategic population
uptown area
immediate area
residential area
possible income
practical employment
full employment
gross income
Net income
Federal income
higher area
wide area
metropolitan area
downtown area
personal income
ample area
large population
large population
raw population
Catholic population
religious population
important area
national population
American population
white population
geographic area
cross-sectional area
wooded area
living area
living area
industrial area
large area
useful area
light area
black area
geographical area
minimal income
urban area
steep mountain
civilian population
known mountain
general population
annual income
industrial area
gross income
American population
respectable income
Jewish population
metropolitan area
volcanic mountain
poor area
residential area
personal income
maximum employment
small area
small area
homogeneous population
graduated income
regular employment
open area
Celtic population
Celtic population
free employment
Atlantic are

In [62]:
def reqadj(adjs):
    # return adjs in descending frequency in the given data set. only words with freq >1 are considered
    [awords,freq]=np.unique(adjs, return_counts=True)
    awords = awords[freq>1]
    freq=freq[freq>1]
    freqa=awords[np.argsort(-freq)]
    return freqa
def isadj(word):
    for kk in wn.synsets(word):
        if kk.pos()=='a':
            return True
    return False
def searchsent(sentences,words,topics,fname,doprint=False):
    wsents = [[] for k in range(len(words))]
    tsents = [[] for k in range(len(topics))]
    wbef = [[] for k in range(len(words))]
    for st in sentences:
        if type(st) is not list:
            swords = tokenizer.tokenize(st)
        else:
            swords = st
        for kword in range(len(words)):
            word = words[kword]
            if word in swords:        
                wsents[kword].append(swords)
                widx=swords.index(word)
                if widx>0: #if it's the beginning of sentence, skip this sentence
                    wordbef = swords[widx-1]
                    if wordbef not in stopwords.words("english"):               
                        """try:
                            wordpos = wn.synsets(wordbef)[0].pos()
                        except:
                            continue"""
                        if isadj(wordbef):
                            if doprint:
                                print(wordbef+' '+word)
                            wbef[kword].append(wordbef)

        for kt in range(len(topics)): # todo: search for broader topics to look for other critical nouns
            topic = topics[kt]
            if topic in swords:
                tsents[kt].append(swords)
    pickle.dump([wsents,tsents],open(fname,'wb'))
    return [wbef,wsents,tsents]

In [50]:
# obsolete: webtext is not a good corpora
sentns = []
for fileid in webtext.fileids():
    sentns.append(webtext.sents(fileid))
websents = [] # join them together
for cs in sentns:
    for s in cs:
        websents.append(s)
fname='webtext_sents.p'
results=searchsent(websents[:1111],words,topics, fname)

empty area
content area
content area
black area
content area
content area
Empty area
